In [2]:
import torch
import numpy as np
from sketched_optimizer import SketchedSGD, SketchedSum, SketchedModel

In [3]:
X = torch.randn(1000, 3)
w = torch.tensor([[.1],[.2],[.3]])

In [4]:
y = torch.matmul(X,w)

In [5]:
y.shape

torch.Size([1000, 1])

In [6]:
model = torch.nn.Sequential(torch.nn.Linear(3, 1))
model = SketchedModel(model)

In [7]:
opt = torch.optim.SGD(model.parameters(), lr=0.0001)
opt = SketchedSGD(opt, k=1, accumulateError=True, p1=0, p2=2)

In [8]:
summer = SketchedSum(opt, c=20, r=5, numWorkers=4)

device cpu
D: 4
sketchMask.sum(): 3
making sketches


In [9]:
yPred = model(X)
loss = (yPred - y)**2
loss = summer(loss)
loss.item()

437.2314147949219

In [10]:
for i in range(20):
    opt.zero_grad()

    yPred = model(X)
    loss = (yPred - y)**2

    loss = summer(loss)

    print("[{}] Loss: {}".format(i, loss.item()))

    loss.backward()

    opt.step()

[0] Loss: 437.2314147949219
[1] Loss: 297.7415771484375
[2] Loss: 207.2788848876953
[3] Loss: 169.70272827148438
[4] Loss: 79.09648895263672
[5] Loss: 69.72575378417969
[6] Loss: 34.21181106567383
[7] Loss: 25.62523078918457
[8] Loss: 11.616199493408203
[9] Loss: 7.967192649841309
[10] Loss: 6.996339797973633
[11] Loss: 3.413329601287842
[12] Loss: 2.564345359802246
[13] Loss: 1.1465332508087158
[14] Loss: 0.77061927318573
[15] Loss: 0.65976881980896
[16] Loss: 0.29584410786628723
[17] Loss: 0.19905538856983185
[18] Loss: 0.16993792355060577
[19] Loss: 0.07712558656930923


In [11]:
for name, parameter in model.model.named_parameters():
    print(f'{name}: {parameter.detach().numpy()}')

0.weight: [[0.09954208 0.19264236 0.2991756 ]]
0.bias: [-0.00363863]


In [161]:
#state_dict = model.state_dict()
#state_dict['0.weight'] = torch.tensor([[0.11,0.22,0.33]])
#state_dict['0.bias'] = torch.tensor([0.0])
#model.load_state_dict(state_dict)

<All keys matched successfully>